In [2]:
import os
import copy
import csv
import shutil

import matplotlib.pyplot as plt

import open3d as o3d
import numpy as np
import nibabel as nib
import glob
from util import *
%matplotlib inline

In [ ]:
# sub_dict = read_dict_from_csv(rf"G:\WJ\OASIS\normal.csv", flag="") # 0:ID, 1:L_tail, 2:R_tail

tail_tail = np.load(rf"temp_meshes\lv_hippo_tail_tail.npy")
tail_notail = np.load(rf"temp_meshes\lv_hippo_tail_notail.npy")
notail_notail = np.load(rf"temp_meshes\lv_hippo_notail_notail.npy")
notail_tail = np.load(rf"temp_meshes\lv_hippo_notail_tail.npy")

tail_tail_pcd = voxel_to_bd_pcd((tail_tail!=0).astype(int))
tail_notail_pcd = voxel_to_bd_pcd((tail_notail!=0).astype(int))
notail_notail_pcd = voxel_to_bd_pcd((notail_notail!=0).astype(int))
notail_tail_pcd = voxel_to_bd_pcd((notail_tail!=0).astype(int))
 
tail_L = o3d.io.read_triangle_mesh(rf"temp_meshes\hippo_lv_cuttail_temp_mesh_L_coord.obj")
notail_L = o3d.io.read_triangle_mesh(rf"temp_meshes\hippo_lv_cuttail_temp_mesh_L_coord.obj")
with open (r'temp_meshes\cuttail_temp_L.pkl', "rb") as f:
    temp_data = pickle.load(f)
    vert = temp_data['cuttail_vert']
    lv_tri = temp_data['cuttail_lv']
    hippo_tri = temp_data['cuttail_hippo']

# tri = np.concatenate([lv_t`ri,hippo_tri])
# print(tri.shape)
# notail_L = o3d.geometry.TriangleMesh(vertices = o3d.utility.Vector3dVector(vert), triangles = o3d.utility.Vector3iVector((tri)))

hippo_tri0 =hippo_tri.copy()
hippo_tri[:,[0,1]] = hippo_tri[:,[1,0]]
lv_tri0 = lv_tri.copy()
lv_tri[:,[0,1]] = lv_tri[:,[1,0]]
# o3d.visualization.draw_geometries([notail_L])
hippo_mesh = copy.deepcopy(notail_L)
hippo_mesh.triangles = o3d.utility.Vector3iVector(hippo_tri)
lv_mesh = copy.deepcopy(notail_L)
lv_mesh.triangles = o3d.utility.Vector3iVector(lv_tri)
lv_mesh.paint_uniform_color([1.0, 0.2, 0.5])
hippo_mesh.paint_uniform_color([0.5, 0.5, 1.0])

for pcd in [tail_tail_pcd, tail_notail_pcd, notail_notail_pcd, notail_tail_pcd]:
    pcd.paint_uniform_color([0.5, 0.5, 0.5])

optim_data = glob.glob(rf"G:\WJ\AIBL\normal_tgt\*.pkl")
print(optim_data)
o3d.visualization.draw_geometries([lv_mesh, hippo_mesh])
for ind, file in enumerate(tqdm(optim_data)):
    try:
        # print(file)
        id = file.split("\\")[-1].split("_")[0]

        with open(file, "rb") as f:
            data = pickle.load(f)
        lv = o3d.geometry.PointCloud(points=o3d.utility.Vector3dVector(data['lv']))
        hippo = o3d.geometry.PointCloud(points=o3d.utility.Vector3dVector(data['hippo']))
        tgt_pt = np.concatenate([np.asarray(hippo.points), np.asarray(lv.points)],axis=0)
        lv.paint_uniform_color([1.0, 0.7, 0.4])
        hippo.paint_uniform_color([0.5, 0.8 ,0.8])
        # print(glob.glob(rf"G:\WJ\OASIS\notail_L_scaler\out\{id}\smallest_*.npy")[0])
        optim_vert = np.load(glob.glob(rf"G:\WJ\AIBL\optim\aibl_normal\out\{id}\smallest_*.npy")[0])[0]
        print(optim_vert[0])
        
        # print(optim_vert.shape)
        # optim_vert = optim_vert - np.min(optim_vert, axis=0) + np.min(tgt_pt, axis=0)
        hippo_pt = np.asarray(hippo.points)
        lv_pt = np.asarray(lv.points)
        hippo_pt[:,0] -= lv_pt[:,0].min()
        # tex1_pt[:,0] -= lv_pt[:,0].min()
        # tex2_pt[:,0] -= lv_pt[:,0].min()
        # tex3_pt[:,0] -= lv_pt[:,0].min()
        vert[:,0] = vert[:,0] - vert[:,0].min()
        lv_pt[:,0] -= lv_pt[:,0].min()
        
        hippo_pt[:,1] -= lv_pt[:,1].max()
        # tex1_pt[:,1] -= lv_pt[:,1].max()
        # tex2_pt[:,1] -= lv_pt[:,1].max()
        # tex3_pt[:,1] -= lv_pt[:,1].max()
        vert[:,1] = vert[:,1] - vert[:,1].max()
        lv_pt[:,1] -= lv_pt[:,1].max()
        
        tgt_vert= np.concatenate((lv_pt, hippo_pt), axis=0)
        hippo_pt[:,2] = hippo_pt[:,2] - (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2
        # tex1_pt[:,2] = tex1_pt[:,2] - (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2
        # tex2_pt[:,2] = tex2_pt[:,2] - (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2
        # tex3_pt[:,2] = tex3_pt[:,2] - (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2
        lv_pt[:,2] = lv_pt[:,2] - (tgt_vert[:,2].max()+tgt_vert[:,2].min())/2
        vert[:,2] = vert[:,2] - (vert[:,2].max()+vert[:,2].min())/2
        # print(optim_vert.shape)
        
        # print(tgt_pt.shape)
        
        lv_mesh.vertices = o3d.utility.Vector3dVector(optim_vert)
        hippo_mesh.vertices = o3d.utility.Vector3dVector(optim_vert)
        o3d.visualization.draw_geometries([lv_mesh, hippo_mesh, lv, hippo], window_name = id)
        print(id)
    except:
        continue


['G:\\WJ\\AIBL\\normal_tgt\\I1055792_cuttail_L_opt_data.pkl', 'G:\\WJ\\AIBL\\normal_tgt\\I1075917_cuttail_L_opt_data.pkl', 'G:\\WJ\\AIBL\\normal_tgt\\I1078865_cuttail_L_opt_data.pkl', 'G:\\WJ\\AIBL\\normal_tgt\\I1080022_cuttail_L_opt_data.pkl', 'G:\\WJ\\AIBL\\normal_tgt\\I1091356_cuttail_L_opt_data.pkl', 'G:\\WJ\\AIBL\\normal_tgt\\I131141_cuttail_L_opt_data.pkl', 'G:\\WJ\\AIBL\\normal_tgt\\I132872_cuttail_L_opt_data.pkl', 'G:\\WJ\\AIBL\\normal_tgt\\I132873_cuttail_L_opt_data.pkl', 'G:\\WJ\\AIBL\\normal_tgt\\I137666_cuttail_L_opt_data.pkl', 'G:\\WJ\\AIBL\\normal_tgt\\I138713_cuttail_L_opt_data.pkl', 'G:\\WJ\\AIBL\\normal_tgt\\I171924_cuttail_L_opt_data.pkl', 'G:\\WJ\\AIBL\\normal_tgt\\I195628_cuttail_L_opt_data.pkl', 'G:\\WJ\\AIBL\\normal_tgt\\I246671_cuttail_L_opt_data.pkl', 'G:\\WJ\\AIBL\\normal_tgt\\I254635_cuttail_L_opt_data.pkl', 'G:\\WJ\\AIBL\\normal_tgt\\I254674_cuttail_L_opt_data.pkl', 'G:\\WJ\\AIBL\\normal_tgt\\I256986_cuttail_L_opt_data.pkl', 'G:\\WJ\\AIBL\\normal_tgt\\I443971

  0%|          | 0/150 [00:00<?, ?it/s]

[14.798552 -5.816826 16.055862]
I1075917
[17.846712  -4.2003207 18.731564 ]
I444577
[19.884865  -5.5006814 18.553234 ]
I444602
[18.5605   -6.742801 18.887703]
I445688
[17.715738  -3.9122448 15.629856 ]
I445731
[14.478958  -5.0597525 16.3296   ]
I446757
[16.76415   -4.9999385 19.26002  ]
I450352
[17.992771  -5.9375753 21.381567 ]
I450353
[16.361732 -5.834658 20.676308]
I450384
[16.326237 -4.852549 20.680977]
I450389
[16.392475  -3.6993573 15.096304 ]
I450876
[21.516537  -4.8664346 20.462063 ]
I450933
[19.358448  -5.7321897 21.424238 ]
I451149
[17.129856  -4.7505465 18.289598 ]
I451169
[18.654167 -4.736556 19.909603]
I451190
[21.908     -6.1414533 22.313513 ]
I451388
[17.995455  -4.9635906 17.731894 ]
I451419
[16.272652  -5.1906185 20.104242 ]
I453061
[16.633694  -5.7342443 19.564219 ]
I453076
[17.94965   -4.8074017 18.33214  ]
I473581


In [16]:
lv_mesh.vertices = o3d.utility.Vector3dVector(optim_vert)
o3d.visualization.draw_geometries([lv_mesh])

In [3]:
print(file)
id = file.split("\\")[-1].split("_")[1]
with open(file, "rb") as f:
    data = pickle.load(f)
lv = o3d.geometry.PointCloud(points=o3d.utility.Vector3dVector(data['lv']))
hippo = o3d.geometry.PointCloud(points=o3d.utility.Vector3dVector(data['hippo']))
tgt_pt = np.concatenate([np.asarray(hippo.points), np.asarray(lv.points)],axis=0)
lv.paint_uniform_color([1.0, 0.7, 0.4])
hippo.paint_uniform_color([0.5, 0.8 ,0.8])
optim_data = glob.glob(rf"G:\WJ\OASIS\notail_L_scaler\out\{id}\smallest_*.npy")[0]
optim_vert = np.load(optim_data)[0]

print(optim_vert.shape)
optim_vert = optim_vert - np.min(optim_vert, axis=0) + np.min(tgt_pt, axis=0)
print(optim_vert.shape)

print(tgt_pt.shape)

lv_mesh.vertices = o3d.utility.Vector3dVector(optim_vert)
hippo_mesh.vertices = o3d.utility.Vector3dVector(optim_vert)
o3d.visualization.draw_geometries([lv_mesh, hippo_mesh, lv, hippo])

NameError: name 'file' is not defined

In [ ]:
import numpy as np
import trimesh
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh

def compute_laplacian(mesh):
    """
    Compute the cotangent Laplacian matrix for a given mesh.
    """
    vertices = mesh.vertices
    faces = mesh.faces

    n_vertices = len(vertices)
    I, J, W = [], [], []

    for face in faces:
        for i in range(3):
            i1, i2, i3 = face[i], face[(i + 1) % 3], face[(i + 2) % 3]
            v1, v2, v3 = vertices[i1], vertices[i2], vertices[i3]
            
            # Compute cotangent weights
            cot12 = np.dot(v3 - v1, v3 - v2) / np.linalg.norm(np.cross(v3 - v1, v3 - v2))
            cot23 = np.dot(v1 - v2, v1 - v3) / np.linalg.norm(np.cross(v1 - v2, v1 - v3))
            cot31 = np.dot(v2 - v3, v2 - v1) / np.linalg.norm(np.cross(v2 - v3, v2 - v1))
            
            I.extend([i1, i2, i2, i3, i3, i1])
            J.extend([i2, i1, i3, i2, i1, i3])
            W.extend([cot12, cot12, cot23, cot23, cot31, cot31])
    
    # Construct sparse matrix
    L = csr_matrix((W, (I, J)), shape=(n_vertices, n_vertices))
    L = -0.5 * (L + L.T)  # Symmetrize
    diagonal = -np.array(L.sum(axis=1)).flatten()
    L.setdiag(diagonal)
    return L

def compute_spectral_decomposition(mesh, num_eigenvalues=10):
    """
    Compute the spectral decomposition (eigenvalues and eigenvectors) of the mesh Laplacian.
    """
    L = compute_laplacian(mesh)
    eigenvalues, eigenvectors = eigsh(L, k=num_eigenvalues, which='SM')
    return eigenvalues, eigenvectors

def reconstruct_mesh(mesh, eigenvectors, num_eigenvectors=None):
    """
    Reconstruct the mesh using a specified number of eigenvectors.
    If `num_eigenvectors` is None, all eigenvectors will be used.
    """
    if num_eigenvectors is None:
        num_eigenvectors = eigenvectors.shape[1]  # Use all eigenvectors

    # Use the first `num_eigenvectors` eigenvectors
    reduced_basis = eigenvectors[:, :num_eigenvectors]

    # Compute the mass matrix (area-weighted diagonal matrix for vertices)
    mass_matrix = np.eye(len(mesh.vertices))  # Assume uniform weighting
    for face in mesh.faces:
        for i in range(3):
            mass_matrix[face[i], face[i]] += 1  # Simple weighting by valence

    # Normalize the eigenvectors to match the mesh scale
    normalization_factors = np.sqrt(np.diag(reduced_basis.T @ mass_matrix @ reduced_basis))
    reduced_basis /= normalization_factors

    # Project the original vertices onto the reduced basis
    original_vertices = mesh.vertices
    coefficients = reduced_basis.T @ mass_matrix @ original_vertices
    print(coefficients.shape, '!!')
    print(coefficients)
    coefficients[0] *=0.8
    coefficients[1] *=0.8
    projected = reduced_basis @ coefficients

    # Create a new mesh with the reconstructed vertices
    reconstructed_mesh = trimesh.Trimesh(vertices=projected, faces=mesh.faces)
    return reconstructed_mesh

def main():
    # Load a 3D mesh
    mesh = trimesh.load(r'D:\IdeaProjects\IITP_synth\lv-parametric-modelling\ipynb\MICCAI-LV\temp_meshes\lv_tail_temp_mesh_L_wt.obj')

    # Compute spectral decomposition
    num_eigenvalues = 100
    eigenvalues, eigenvectors = compute_spectral_decomposition(mesh, num_eigenvalues)

    # Reconstruct the mesh using the first 5 eigenvectors
    num_eigenvectors = 100
    reconstructed_mesh = reconstruct_mesh(mesh, eigenvectors, num_eigenvectors)

    # Visualize the original and reconstructed meshes
    print("Original Eigenvalues:", eigenvalues)
    print("Showing original mesh...")
    mesh.show()
    new_mesh = o3d.geometry.TriangleMesh()
    new_mesh.vertices = o3d.utility.Vector3dVector(reconstructed_mesh.vertices)
    new_mesh.triangles = o3d.utility.Vector3iVector(reconstructed_mesh.faces)
    orig_mesh = o3d.geometry.TriangleMesh()
    vert= np.asarray(mesh.vertices)
    vert[:,0] += 30
    orig_mesh.vertices = o3d.utility.Vector3dVector(vert)
    orig_mesh.triangles = o3d.utility.Vector3iVector(mesh.faces)
    
    o3d.visualization.draw_geometries([new_mesh, orig_mesh], mesh_show_wireframe=True, point_show_normal=True)
    print(f"Showing reconstructed mesh using the first {num_eigenvectors} eigenvectors...")
    reconstructed_mesh.show()

if __name__ == "__main__":
    main()


In [2]:
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg
import trimesh

def compute_laplacian_eigen(mesh, k=50):
    """
    Compute the Laplacian eigen decomposition of a given mesh.
    
    Parameters:
        mesh (trimesh.Trimesh): The input mesh.
        k (int): Number of eigenvectors to compute.
    
    Returns:
        (numpy.ndarray, numpy.ndarray): Eigenvalues and eigenvectors of the mesh Laplacian.
    """
    vertices = mesh.vertices
    faces = mesh.faces
    
    # Compute Cotangent Laplacian using trimesh
    L = trimesh.smoothing.laplacian_calculation(mesh)
    
    # Compute Mass Matrix (Diagonal with Voronoi Areas)
    A = trimesh.triangles.mass_matrix(mesh)
    
    # Solve the generalized eigenvalue problem Lx = λAx
    evals, evecs = sp.linalg.eigsh(L, k=k, M=A, sigma=0)

    return evals, evecs


# Example usage
mesh = trimesh.load(r"D:\IdeaProjects\IITP_synth\lv-parametric-modelling\ipynb\MICCAI-LV\temp_meshes\lv_tail_temp_mesh_L_wt.obj") # 메쉬 파일 경로를 넣어주세요.
evals, evecs = compute_laplacian_eigen(mesh, k=100)

print("Eigenvalues:", evals)
print("Eigenvectors shape:", evecs.shape)


AttributeError: module 'trimesh.triangles' has no attribute 'mass_matrix'